# Lab 5 — Data Profiling Takeaways (HW4_1.csv)

This notebook loads `HW4_1.csv`, shows profiling-style summaries, and provides a place to write **five findings** relevant to your analysis.


## 1) Load data

In [ ]:
import pandas as pd
import numpy as np

# If your CSV is in the repo root, this works:
df = pd.read_csv("HW4_1.csv")

df.head()


,year,dist,passen,fare,bmktshr,lfare,lpassen,ldist,lbmktshr,fare_str
0,1999,528,336,113,0.8262,4.727388,5.817111,6.269096,-0.190918,$113
1,1999,861,204,115,0.7319,4.744932,5.318120,6.758094,-0.312111,$115
2,1999,852,244,229,0.7998,5.433722,5.497168,6.747587,-0.223394,$229
3,1999,724,224,229,0.4334,5.433722,5.411646,6.584791,-0.836094,$229
4,1999,1073,494,121,0.5008,4.795791,6.202536,6.978214,-0.691548,$121


## 2) Basic overview

In [4]:
df.shape, df.columns.tolist()


((1149, 10),
 ['year',
  'dist',
  'passen',
  'fare',
  'bmktshr',
  'lfare',
  'lpassen',
  'ldist',
  'lbmktshr',
  'fare_str'])

In [5]:
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1149 non-null   int64  
 1   dist      1149 non-null   int64  
 2   passen    1149 non-null   int64  
 3   fare      1149 non-null   int64  
 4   bmktshr   1149 non-null   float64
 5   lfare     1149 non-null   float64
 6   lpassen   1149 non-null   float64
 7   ldist     1149 non-null   float64
 8   lbmktshr  1149 non-null   float64
 9   fare_str  1149 non-null   str    
dtypes: float64(5), int64(4), str(1)
memory usage: 89.9 KB


## 3) Missing values

In [6]:
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]


Series([], dtype: int64)

In [7]:
(df.isna().mean().sort_values(ascending=False) * 100).head(10)


year        0.0
dist        0.0
passen      0.0
fare        0.0
bmktshr     0.0
lfare       0.0
lpassen     0.0
ldist       0.0
lbmktshr    0.0
fare_str    0.0
dtype: float64

## 4) Summary statistics

In [8]:
df.describe(include='all').T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
year,1149.0,NaN,NaN,NaN,1999.0,0.0,1999.0,1999.0,1999.0,1999.0,1999.0
dist,1149.0,NaN,NaN,NaN,989.744996,612.031331,95.0,505.0,861.0,1304.0,2724.0
passen,1149.0,NaN,NaN,NaN,657.832028,842.37215,4.0,215.0,368.0,730.0,7340.0
fare,1149.0,NaN,NaN,NaN,177.970409,74.532229,55.0,124.0,168.0,224.0,492.0
bmktshr,1149.0,NaN,NaN,NaN,0.604869,0.195853,0.1605,0.4659,0.5915,0.7434,1.0
lfare,1149.0,NaN,NaN,NaN,5.09185,0.433383,4.007333,4.820282,5.123964,5.411646,6.198479
lpassen,1149.0,NaN,NaN,NaN,6.037844,0.905268,1.386294,5.370638,5.908083,6.593045,8.901094
ldist,1149.0,NaN,NaN,NaN,6.696482,0.659533,4.553877,6.224558,6.758094,7.173192,7.909857
lbmktshr,1149.0,NaN,NaN,NaN,-0.560911,0.353852,-1.829461,-0.763784,-0.525093,-0.296521,0.0
fare_str,1149,379,$229,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5) Quick sanity checks (ranges / outliers)

In [9]:
cols = ['fare', 'passen', 'dist', 'bmktshr']
df[cols].agg(['min', 'max', 'mean', 'median', 'std']).T


,min,max,mean,median,std
fare,55.0000,492.0,177.970409,168.0000,74.532229
passen,4.0000,7340.0,657.832028,368.0000,842.372150
dist,95.0000,2724.0,989.744996,861.0000,612.031331
bmktshr,0.1605,1.0,0.604869,0.5915,0.195853


In [10]:
df[cols].apply(lambda s: pd.concat([s.nsmallest(5), s.nlargest(5)])).head(10)


,fare,passen,dist,bmktshr
39,NaN,NaN,125.0,0.9989
79,NaN,6793.0,NaN,NaN
110,NaN,4.0,NaN,NaN
112,NaN,4.0,NaN,NaN
114,NaN,NaN,NaN,1.0000
123,NaN,NaN,152.0,NaN
170,60.0,NaN,NaN,NaN
173,NaN,NaN,NaN,0.1605
228,NaN,7096.0,NaN,NaN
238,NaN,NaN,2642.0,NaN


## 6) Check `fare_str` can be parsed to numbers

In [11]:
def parse_dollars(amounts: pd.Series) -> pd.Series:
    dollar_str = (
        amounts.astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .str.strip()
    )
    return dollar_str.astype(float).round(2)

parsed_fare = parse_dollars(df['fare_str'])
parsed_fare.head()


0    113.0
1    115.0
2    229.0
3    229.0
4    121.0
Name: fare_str, dtype: float64

In [12]:
diff = (parsed_fare - df['fare'].astype(float)).abs()
diff.describe()


count    1149.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64

In [13]:
(diff > 0.01).sum(), len(diff)


(np.int64(0), 1149)

## 7) Check derived log columns (consistency)

In [14]:
checks = {
    'lfare vs log(fare)': np.allclose(df['lfare'], np.log(df['fare']), atol=1e-6),
    'lpassen vs log(passen)': np.allclose(df['lpassen'], np.log(df['passen']), atol=1e-6),
    'ldist vs log(dist)': np.allclose(df['ldist'], np.log(df['dist']), atol=1e-6),
    'lbmktshr vs log(bmktshr)': np.allclose(df['lbmktshr'], np.log(df['bmktshr']), atol=1e-6),
}
checks


{'lfare vs log(fare)': True,
 'lpassen vs log(passen)': True,
 'ldist vs log(dist)': True,
 'lbmktshr vs log(bmktshr)': True}

## 8) Five takeaways (writeup)

Fill in **five** findings from the profiling above that matter for your analysis.

1. **Finding 1:**  No missing values in key variables.
All main variables (fare, passen, dist, bmktshr, and log columns) have zero missing values. This means the dataset is complete and ready for regression analysis without additional imputation.
2. **Finding 2:**  Market share is within the valid range.
bmktshr ranges from 0.1605 to 1.0. All values fall between 0 and 1, which is consistent with how market share should behave.
3. **Finding 3:**  Ticket prices and distance show substantial variation.
Fare ranges from 55 to 492, and distance ranges from 95 to 2724 miles. This wide spread suggests meaningful variation across routes, which is useful for explaining price differences.
4. **Finding 4:**  Passenger counts are highly skewed.
Passenger numbers range from 4 to 7340, with a large standard deviation. This indicates strong variation across markets, and suggests that log transformation (lpassen) is appropriate.
5. **Finding 5:**  Derived log variables are correctly calculated.
All log variables (lfare, lpassen, ldist, lbmktshr) exactly match the natural log of their original variables. This confirms internal consistency and that no transformation errors occurred.
